## 1. Setting Up Blender
To begin, ensure you have Blender installed on your system. You can download it from [blender.org](https://www.blender.org/download/).

Download or create your own HDRI's (environment textures) to use inside Blender for random backgrounds.
[Download Free, CC0, High Resolution HDRI's](https://polyhaven.com/hdris) (Use HDR format)

Additionally, set up your Python environment with necessary libraries such as `cv2`, `ultralytics`.


Finlay If you have a dedicated GPU in your system, make sure to set up Blender and pytorch to use it for rendering and model training.

Use the cell below to check your environment and make sure all the packages are working properly.


In [ ]:
# Install required packages
# Note: Run this in a Jupyter environment with internet access

# YOLOv11 (Ultralytics)
%pip install ultralytics --quiet

# OpenCV for image processing
%pip install opencv-python --quiet


# Install torch with CUDA support if a compatible GPU is available
# https://pytorch.org/get-started/locally/


# Blender's Python API is included in Blender itself.
# If you want to run Blender scripts from outside Blender, you need to call Blender in background mode:
# blender --background --python your_script.py

# Check versions
import sys
import cv2
import ultralytics
import os
import torch
from ultralytics import YOLO
import numpy as np

print("Python version:", sys.version)
print("OpenCV version:", cv2.__version__)
print("Ultralytics YOLO version:", ultralytics.__version__)
print("PyTorch version:", torch.__version__)


In [ ]:
# Check if GPU is available for PyTorch training
# This is important for performance, especially with large models and datasets
# For more information, visit: https://pytorch.org/docs/stable/notes/cuda.html
# If you encounter issues, ensure that your CUDA drivers are correctly installed and compatible with your PyTorch version.
print("Is GPU available for PyTorch:")
torch.cuda.is_available()

## 2. Convert Images to Grayscale and organize them for YOLO
To force the model to rely less on color data, we can convert the rendered RGB images to grayscale using OpenCV.

Use the cell below to perform the conversion.


In [ ]:
# Folder containing image data from Blender renders
image_data_folder = "Path/To/Blender/Output/Folder/Images"

# Output folder for grayscale images
output_folder = "Path/To/Folder/Where/To/Store/Training/Data"

# Create output folder and subfolders if they don't exist
os.makedirs(output_folder, exist_ok=True)
os.makedirs(os.path.join(output_folder, "Train"), exist_ok=True)
os.makedirs(os.path.join(output_folder, "Val"), exist_ok=True)

# Convert all images in the image data folder to grayscale and save to output folder, train, and val subfolders
image_files = [f for f in os.listdir(image_data_folder) if f.lower().endswith('.jpg')]
image_files.sort()  # for reproducibility

# Split into training and validation sets (90% train, 20% val)
num_val = max(1, int(0.2 * len(image_files)))
val_files = image_files[:num_val]
train_files = image_files[num_val:]

train_folder = os.path.join(output_folder, "Train")
val_folder = os.path.join(output_folder, "Val")

for fname in train_files:
    img = cv2.imread(os.path.join(image_data_folder, fname))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(os.path.join(train_folder, fname), gray)

for fname in val_files:
    img = cv2.imread(os.path.join(image_data_folder, fname))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(os.path.join(val_folder, fname), gray)

## 3. Generating Annotations in YOLOv11 Format
Once images are rendered, we need to generate bounding box annotations in YOLOv11 format. This format includes class ID and normalized coordinates for each object.

Use the cell below to generate annotation files.

In this example we generate annotation files for YOLOv11 [Object Detection](https://docs.ultralytics.com/tasks/detect/)


In [ ]:
# Get train/val split from grayscale images
train_files = set(os.listdir(train_folder))
val_files = set(os.listdir(val_folder))

# Define range of red color in HSV
lower_red = np.array([0, 50, 50])
upper_red = np.array([10, 255, 255])

mask_images_folder = "Path/To/Blender/Output/Folder/Masks"

for filename in os.listdir(mask_images_folder):
    if filename.endswith(".jpg"):
        img = cv2.imread(os.path.join(mask_images_folder, filename))

        height, width, _ = img.shape

        # Convert BGR to HSV
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        # Threshold the HSV image to get only red colors
        mask = cv2.inRange(hsv, lower_red, upper_red)

        # Find contours in the binary image
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # This assumes the largest contour is the object of interest and you only have one object per image
        if contours:
            contour = max(contours, key=cv2.contourArea)

            # Find the bounding box of the largest contour
            x, y, w, h = cv2.boundingRect(contour)

            # Calculate normalized bounding box coordinates
            x_center = (x + w/2) / img.shape[1]
            y_center = (y + h/2) / img.shape[0]
            width = w / img.shape[1]
            height = h / img.shape[0]

            # Draw the contour on the original image for visualization
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.imshow('Contour', img)
            cv2.waitKey(2)  # Display each image for 2 ms

            # Save annotation in YOLOv11 format
            txt_folder = ""
            # Determine if the image is in the training or validation set
            if filename in train_files:
                txt_folder = train_folder
            elif filename in val_files:
                txt_folder = val_folder
            
            with open(os.path.join(txt_folder, filename.replace('.jpg', '.txt')), 'w') as f:
                # Class ID is 0 for single-class segmentation
                class_id = 0
                
                # Write class ID and points to file
                # YOLOv11 instance segmentation format: class_id x_center y_center width height
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")


cv2.destroyAllWindows()

# Create YOLOv11 YAML configuration file, change the name of the class if needed (from coral to algae, cone, etc.)
yaml_content = f"""
train: {train_folder}
val: {val_folder}
nc: 1
names: ['coral']
"""

yaml_path = os.path.join(output_folder, "yolov11_config.yaml")
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"YOLOv11 configuration file saved to {yaml_path}")

## 5. Training YOLOv11 Object Detection Model
With the dataset and annotations ready, we can now train a YOLOv11 object detection model. This step involves configuring the model, loading the dataset, and running the training process.

Use the cell below to initiate training.

For more info visit the [ultralytics website](https://docs.ultralytics.com/tasks/detect/).


In [ ]:
# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)

# Train the model
device = 0 if torch.cuda.is_available() else "cpu"  # device=0 for GPU, "cpu" for CPU
results = model.train(data=yaml_path, epochs=400, imgsz=640, device=device)

## 6. Using the Trained Model for Detection
After training, we can use the model to detect objects in new images. This step involves loading the trained weights and running predictions on test images.

Use the cell below to perform prediction on test images.


In [ ]:
# Load custom model
model = YOLO("runs/detect/train/weights/best.pt")

# Load test image
test_image = cv2.imread("Path/to/image.jpg", cv2.IMREAD_GRAYSCALE)
test_image = cv2.cvtColor(test_image, cv2.COLOR_GRAY2BGR)

# Use the custom model
model.predict(source=test_image, save=True, show=True, conf=0.1, save_txt=False)